In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Вам предоставлен набор данных, первоначально полученный в Национальном институте диабета, болезней органов пищеварения и почек. 

Этот набор данных создан для того, чтобы на основе определённых диагностических измерений предсказать, есть ли у пациента диабет. 

На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года индейского происхождения Пима.

Прочитаем наши данные и выведем первые пять строк таблицы:

In [2]:
diabetes = pd.read_csv('data/diabetes_data.csv')
display(diabetes.head())
diabetes.shape[1]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


10

Задание 8.1

Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [3]:
diabetes_dedupped = diabetes.drop_duplicates()
display(f'Результирующее число записей: {diabetes_dedupped.shape[0]}')

'Результирующее число записей: 768'

Задание 8.2

Найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).



In [4]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes_dedupped.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes_dedupped[col].nunique() / diabetes_dedupped[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

diabetes_dedupped = diabetes_dedupped.drop(['Gender'], axis=1)

display(diabetes_dedupped.shape[1])

Gender: 100.0% одинаковых значений


9

Задание 8.3

Попробуйте найти пропуски в данных с помощью метода isnull()

In [5]:
# функция заменяющая значение 0 на nan
def nan_function(x):
    return np.nan if x == 0 else x

#применение функции к нужным столбцам
diabetes_dedupped["Glucose"] = diabetes_dedupped["Glucose"].apply(nan_function)
diabetes_dedupped["BloodPressure"] = diabetes_dedupped["BloodPressure"].apply(nan_function)
diabetes_dedupped["SkinThickness"] = diabetes_dedupped["SkinThickness"].apply(nan_function)
diabetes_dedupped["Insulin"] = diabetes_dedupped["Insulin"].apply(nan_function)
diabetes_dedupped["BMI"] = diabetes_dedupped["BMI"].apply(nan_function)

display(diabetes_dedupped.head(3))

# расчет доли пропусков в столбцах
diabetes_dedupped.isnull().mean().round(2).sort_values(ascending=False)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,190.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,NaN,35.7,0.148,21,0
2,2,108.0,64.0,NaN,NaN,30.8,0.158,21,0


Insulin                     0.49
SkinThickness               0.30
BloodPressure               0.05
Glucose                     0.01
BMI                         0.01
Pregnancies                 0.00
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

Задание 8.4

Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [6]:
#создаем копию исходной таблицы
diabetes_droped = diabetes_dedupped.copy()
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = diabetes_droped.shape[0]*0.7
#удаляем столбцы (признаки), в которых более 30% (100-70) пропусков
diabetes_droped = diabetes_droped.dropna(thresh=thresh, axis=1)
#отображаем результирующую долю пропусков
print(diabetes_droped.shape[1])

8


Задание 8.5

Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

In [7]:
#удаляем записи, в которых есть хотя бы 1 пропуск
m = diabetes_droped.shape[1]
diabetes_droped = diabetes_droped.dropna(thresh=m-2, axis=0)
display(diabetes_droped.shape[0])

761

Задание 8.6

В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.

In [9]:
# опредиляем количество пропусков 
null_data = diabetes_droped.isnull().sum()
# создаем список из столбцов с пропусками
cols = null_data[null_data>0].index
# цикл по каждому столбцу с пропусками
for col in cols:
    # заменяем содержание пропусков в столбцах на медианное значение
    diabetes_droped[col] = diabetes_droped[col].fillna(diabetes_droped[col].median())
print(diabetes_droped['SkinThickness'].mean().round(1))

29.1


Задание 8.7

Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

In [14]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    left = left * iqr
    right = right * iqr
    lower_bound = quartile_1 - left
    upper_bound = quartile_3 + right
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(diabetes_droped, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


Задание 8.8

Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

In [21]:
def outliers_z_score_mod(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score_mod(diabetes_droped, 'SkinThickness')
print(f'выбросов : {outliers.shape[0]}')

выбросов : 4


Задание 8.9

In [23]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_iqr_mod(diabetes_droped, 'DiabetesPedigreeFunction')
outliers_log, _ = outliers_iqr_mod(diabetes_droped, 'DiabetesPedigreeFunction', log_scale=True)
print(outliers.shape[0] - outliers_log.shape[0])



29
